[Source](https://petewarden.com/2016/02/28/tensorflow-for-poets/ "Permalink to TensorFlow for Poets « Pete Warden's blog")

# TensorFlow for Poets « Pete Warden's blog

When I first started investigating the world of deep learning, I found it very hard to get started. There wasn't much documentation, and what existed was aimed at academic researchers who already knew a lot of the jargon and background. Thankfully that has changed over the last few years, with a lot more guides and tutorials appearing.

I always loved [EC2 for Poets][1] though, and I haven't seen anything for deep learning that's aimed at as wide an audience. EC2 for Poets is an explanation of cloud computing that removes a lot of the unnecessary mystery by walking anyone with basic computing knowledge step-by-step through building a simple application on the platform. In the same spirit, I want to show how anyone with a Mac laptop and the ability to use the Terminal can create their own image classifier using [TensorFlow][2], without having to do any coding.

I feel very lucky to be a part of building [TensorFlow][3], because it's a great opportunity to bring the power of deep learning to a mass audience. I look around and see so many applications that could benefit from the technology by understanding the images, speech, or text their users enter. The frustrating part is that deep learning is still seen as a very hard topic for product engineers to grasp. That's true at the cutting edge of research, but otherwise it's mostly a holdover from the early days. There's already a lot of great documentation on the [TensorFlow][2] site, but to demonstrate how easy it can be for general software engineers to pick up I'm going to present a walk-through that takes you from a clean OS X laptop all the way to classifying your own categories of images. You'll find written instructions in this post, along with [a screencast showing exactly what I'm doing][4].

### Downloading Images

The rest of this walk-through is based on [the image-retraining example on the TensorFlow site][9]. It shows you how to take your own images organized into folders by category, and use them to quickly retrain the top layer of the Inception image recognition neural network to recognize those categories. To get started, the first thing you need to do is get some example images. To begin, go to the terminal and enter the 'exit' command if you still see the 'root@…' prompt that indicates you're still in the Linux virtual machine.

Then run the following commands to create a new folder in your Downloads directory to hold training images, and download and extract the flower photos:

In [ ]:
!mkdir -p /usr/workdir/tf_files
%cd /usr/workdir/tf_files

In [ ]:
!curl -O http://download.tensorflow.org/example_images/flower_photos.tgz

In [ ]:
!tar xzf flower_photos.tgz

This should end up with a new finder window opening, showing a set of five folders:

![file list](file-list.png)

This means you've successfully downloaded the example flower images. If you look at how they're organized, you should be able to use the same structure with classes you care about, just replacing the folder names with the category labels you're dealing with, and populating them with photos of those objects. [There's more guidance on that process in the tutorial][11].

### Updating the Code

For this example, we need the very latest code since it's just been added. Unfortunately getting it is a little involved, with some use of the source control program [git][12]. I'll walk through the steps below.

Pulling the code requires a default email address, which you can set to anything, since we're not planning on pushing any changes back.

In [13]:
!git config --global user.email "andrew@odewahn.com"
!git config --global user.name "Andrew Odeahn"

Now you should be able to pull the latest source.

In [ ]:
%cd /tensorflow/
!git pull origin master

You'll find yourself in a vim window. Just type ':quit' to exit.

You should now have fully up-to-date code. We want to sync it to a version we know works though, so we'll run this command:

    git checkout 6d46c0b370836698a3195a6d73398f15fa44bcb2
 

### Building the Code

If that worked, the next step is to compile the code. You may notice there's some optimization flags in the command that help speed it up on processors with AVX, which almost all modern OS X machines have.

In [ ]:
%cd /tensorflow/
!bazel build -c opt --copt=-mavx tensorflow/examples/image_retraining:retrain

This part can take five to ten minutes, depending on the speed of your machine, as it's compiling the full source code for TensorFlow. Don't worry if you see a lot of warnings, this is normal (though we're working on reducing them going forward).

### Running the Code

I can now run the retraining process using this command:

In [ ]:
!bazel-bin/tensorflow/examples/image_retraining/retrain \
    --bottleneck_dir=/tf_files/bottlenecks \
    --model_dir=/tf_files/inception \
    --output_graph=/tf_files/retrained_graph.pb \
    --output_labels=/tf_files/retrained_labels.txt \
    --image_dir /tf_files/flower_photos

You'll see a message about downloading the Inception model, and then a long series of messages about creating bottlenecks. There's around 3,700 photos in total to process, and my machine does around 200 a minute, so it takes around twenty minutes in total. If you want to know more about what's happening under the hood while you wait, [you can check out the tutorial for a detailed explanation][13].

I've changed the default /tmp destination for things like the output graph and cached bottlenecks to the shared /tf_files folder, so that the results will be accessible from OS X and will be retained between different runs of the virtual machine.

Once the bottlenecks are cached, it will then go into the training process, which takes another five minutes or so on my laptop. At the end, you should see the last output line giving the final estimated accuracy, which should be around 90%. That means you've trained your classifier to guess the right flower species nine times out of ten when shown a photo!

### Using the Classifier

The training process outputs the retrained graph into /tmp/output_graph.pb, and to test it out yourself you can build another piece of sample code. The [label_image example][14] is a small C++ program that loads in a graph and applies it to a user-supplied image. Give it a try like this:

In [ ]:
!bazel build tensorflow/examples/label_image:label_image && /
    bazel-bin/tensorflow/examples/label_image/label_image /
    --graph=/tf_files/retrained_graph.pb /
    --labels=/tf_files/retrained_labels.txt /
    --output_layer=final_result /
    --image=/tf_files/flower_photos/daisy/21652746_cc379e0eea_m.jpg

You should see a result showing that it identified a daisy in that picture, though because the training process is random you may occasionally have a model that makes a mistak on the image. Try it with some of the other photos to get a feel for how it's doing.

### Next Steps

The first thing you'll probably want to do is train a classifier for objects you care about in your application. This should be as simple as creating a new folder in your Downloads/tf_images directory, putting subfolders full of photos in it, and re-running the classifier commands. You can find more detailed advice on tuning that process [in the tutorial][11].

Finally, you'll want to use this in your own application! The [label_image example][14] is a good template to look at if you can integrate C++ into your product, and we even support running on mobile, so check out t[he Android sample code][15] if you'd like to run on a smart phone.

Thanks for working through this process with me, I hope it's inspired you to think about how you can use deep learning to help your users, and I can't wait to see what you build!

### _Related_

* http://ec2.forpoets.org/
* http://www.tensorflow.org/
* http://www.tensorflow.org
* https://www.youtube.com/watch?v=h7xuEiZjqqo
* https://docs.docker.com/mac/
* https://www.tensorflow.org/versions/r0.7/get_started/os_setup.html#docker-installation
* https://petewarden.files.wordpress.com/2016/02/screen-shot-2016-02-27-at-3-22-15-pm.png?w=550
* https://www.virtualbox.org/wiki/Downloads
* https://www.tensorflow.org/versions/master/how_tos/image_retraining/index.html
* https://petewarden.files.wordpress.com/2016/02/screen-shot-2016-02-27-at-4-07-18-pm.png?w=550
* https://www.tensorflow.org/versions/master/how_tos/image_retraining/index.html#training-on-your-own-categories
* https://git-man-page-generator.lokaltog.net/
* https://www.tensorflow.org/versions/master/how_tos/image_retraining/index.html#bottlenecks
* https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/label_image
* https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android 